# Monte-Carlo Dropout for uncertainty prediction on CIFAR10

We implement Monte-Carlo dropout for uncertainty prediction according to https://arxiv.org/pdf/1506.02142.pdf

Therefore we train a model on CIFAR10 using dropout and then at inference time we keep using dropout to estimate the uncertainty of the network by performing several forward passes through the network for each sample. 

# Imports

In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


# Hyperparameters

In [2]:
batch_size = 128
num_classes = 10
epochs = 50
mc_samples = 10 # number of samples for prediciton

# Load and preprocess CIFAR10

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

img_rows, img_cols = x_train.shape[1], x_train.shape[2]

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


# Build model

In [4]:
def get_model():
    """
    Use only minimalistic model to get some statistics for misclassifications
    """
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(256, kernel_size=(3, 3),
#                      activation='relu',
#                      input_shape=input_shape))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    # This dropout layer stays active during testing phase
    model.add(Lambda(lambda x: K.dropout(x, level=0.25)))
    model.add(Dense(512, activation='relu'))
    
    model.add(Lambda(lambda x: K.dropout(x, level=0.5)))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    return model

In [5]:
model = get_model()

In [6]:
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test),
            callbacks=[EarlyStopping(patience=5)])

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 9s 181us/step - loss: 1.9404 - acc: 0.2856 - val_loss: 1.6198 - val_acc: 0.4037
Epoch 2/50
50000/50000 [==============================] - 7s 148us/step - loss: 1.4847 - acc: 0.4625 - val_loss: 1.4173 - val_acc: 0.4908
Epoch 3/50
50000/50000 [==============================] - 7s 149us/step - loss: 1.2910 - acc: 0.5390 - val_loss: 1.3208 - val_acc: 0.5258
Epoch 4/50
50000/50000 [==============================] - 7s 148us/step - loss: 1.1553 - acc: 0.5930 - val_loss: 1.1045 - val_acc: 0.6087
Epoch 5/50
50000/50000 [==============================] - 8s 150us/step - loss: 1.0567 - acc: 0.6276 - val_loss: 1.1552 - val_acc: 0.6018
Epoch 6/50
50000/50000 [==============================] - 7s 150us/step - loss: 0.9805 - acc: 0.6554 - val_loss: 1.0296 - val_acc: 0.6359
Epoch 7/50
50000/50000 [==============================] - 7s 149us/step - loss: 0.9209 - acc: 0.6798 - val_loss: 1.0239 - 

# Evaluate Uncertainty Prediction with Monte Carlo Dropout

Evaluate on test set and compare mean standard deviation of all predictions to mean standard deviation of misclassified samples

In [7]:
def get_predictions_with_uncertainty(model, X):
    
    predictions = []
    for i in range(mc_samples): # can be made more efficient by just forward passing several times through the last layer
        predictions.append(model.predict(X))
    predictions = np.array(predictions)
    
    means = np.mean(predictions, axis=0)
    std = np.std(predictions, axis=0)
    preds = np.argmax(means, axis=1)
    preds_std = np.array([std[i, preds[i]] for i in range(len(preds))])

    return preds, preds_std

In [8]:
preds, stds = get_predictions_with_uncertainty(model, x_test)
labels = np.argmax(y_test, axis=1)

print('Accuracy: ' + str((preds == labels).sum()/len(labels)))

misclassified_mask = labels != preds

print('Average standard deviation of classification: ' + str(np.mean(stds)))
print('Average standard deviation of misclassified samples: ' + str(np.mean(stds[misclassified_mask])))

Accuracy: 0.725
Average standard deviation of classification: 0.12922935
Average standard deviation of misclassified samples: 0.18558362
